# Detecting Faces with and without masks

![](three.png)

In [8]:
from keras.models import load_model
import cv2
import numpy as np

Best model
"model-016.model" 

Epoch 16/20
989/989 [==============================] - 62s 62ms/step - loss: 0.0425 - accuracy: 0.9838 - val_loss: 0.2673 - val_accuracy: 0.9032

In [10]:
#Load model method to load the best model
model = load_model('model-016.model')

face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#2 arrays to predict  face with mask or wihout mask in the detected image
#mask found->0
#wear mask->1

source=cv2.VideoCapture(0)
labels_dict={0:'MASK FOUND',1:'WEAR MASK'}

#The bounding rectangle will be green for WITH MASK and red for WITHOUT MASK

color_dict={0:(0,255,0),1:(0,0,255)}

In [7]:
#inside the infinite the loop we are going to read frame by frame form the camera
#and going to get the frame to this image and do the same as per the training data i.e. converting it into grayscale image
#then going to detect faces with mask and without mask by going through each and every face


while(True):
    #For each and every face going to crop the ROI Region Of Interest 
    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5) 
   

    for (x,y,w,h) in faces:
    
        #get the resized that is divide by 255 to normalize
        #and  reshape it to 4D since the CNN takes the 4D input
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        #Im going to use the best model model numXXX  of epoch YY
        #For that neural network model im going to get the predictions
        result=model.predict(reshaped)
        #next pass the reshaped and resized image..
        #result will be taken here
        #then going to get armax function in numpy 
        #it will give us the column number since we given the axis number 1,so it will return column number with max probability
        #i.e.,p1 is max-> face with mask or p2 is max ->face without mask
        #0->mask found, 1->wear mask

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break

#If user press ESC key loop will terminated and windows will be destroyed.
cv2.destroyAllWindows()
source.release()

![](four.png)